# Imports

In [1]:
import pandas as pd
import sklearn.metrics as metrics
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import sklearn.metrics as metrics
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, date, timedelta
import math
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
from time import sleep
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Reshape, TimeDistributed, LSTM, Lambda, Bidirectional, RepeatVector, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import Callback, EarlyStopping, ProgbarLogger, History
import tensorflow as tf
import keras.backend as K
from IPython.display import display
from tensorflow.keras.metrics import get as metric_get
import re
import talos
from copy import deepcopy, copy
import time
import util
import functools 
import itertools
import networkx as nx
from typing import List, Iterator, Tuple, Dict, Union
import json
import inspect
import os

# Set up GPU:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices: tf.config.experimental.set_memory_growth(device, True)

# Load data
load hospitalisations and trends

In [2]:
# topics considered
list_topics = {
    'Fièvre': '/m/0cjf0',
    'Mal de gorge': '/m/0b76bty',
    #'Dyspnée': '/m/01cdt5',
    #'Agueusie': '/m/05sfr2',
    #'Anosmie': '/m/0m7pl',
    #'Coronavirus': '/m/01cpyy',
    #'Virus': '/m/0g9pc',
    #'Température corporelle humaine': '/g/1213j0cz',
    #'Épidémie': '/m/0hn9s',
    'Symptôme': '/m/01b_06',
    #'Thermomètre': '/m/07mf1',
    #'Grippe espagnole': '/m/01c751',
    #'Paracétamol': '/m/0lbt3',
    #'Respiration': '/m/02gy9_',
    #'Toux': '/m/01b_21'
}

# hospitalisations features given as input
list_hosp_features = [
    'NEW_HOSP',
    #'TOT_HOSP',
    #'TOT_HOSP_log',
    #'TOT_HOSP_pct',
]

europe = False  # if True, use european countries. Otherwise, use french regions and belgium

Additional information: what is the target, should some features remain unscaled?

In [3]:
# target, should be one of the hosp features
target = 'NEW_HOSP'

cumsum = False  # if True, the target will be accumulated over each day

# features that should not be scaled
unscaled = [
    #'NEW_HOSP',
    #'TOT_HOSP',
    #'TOT_HOSP_log',
    #'TOT_HOSP_pct',
    #'Fièvre',
    #'Mal de gorge',
    #'Dyspnée',
    #'Agueusie',
    #'Anosmie',
    #'Coronavirus',
    #'Virus',
    #'Température corporelle humaine',
    #'Épidémie',
    #'Symptôme',
    #'Thermomètre',
    #'Grippe espagnole',
    #'Paracétamol',
    #'Respiration',
    #'Toux',
]

Type of prediction: how many days as input sould be used to predict how many days as output? Should we give a prediction on all days or only on the last?

In [4]:
date_begin = "2020-02-01"
n_forecast = 20
n_samples = 30
predict_one = False

In [5]:
validation_metrics = [metric_get("MeanSquaredError"), metric_get('MeanAbsoluteError'), 
                      metric_get('RootMeanSquaredError')]

url_world = "../data/hospi/world.csv"
url_pop = "../data/population.txt"
url_trends = "../data/trends/model/"
url_hospi_belgium = "../data/hospi/be-covid-hospi.csv"
url_department_france = "france_departements.csv"
url_hospi_france_new = "../data/hospi/fr-covid-hospi.csv"
url_hospi_france_tot = "../data/hospi/fr-covid-hospi-total.csv"
if europe:
    population = util.get_world_population(url_pop)
    renaming = {v: k for k, v in util.european_geocodes.items()}
    geocodes = {k: v for k, v in util.european_geocodes.items() if population[k] > 1_000_000}
    df_hospi = util.hospi_world(url_world, geocodes, renaming, new_hosp=True, date_begin=date_begin)
    augment_population = {k: v/1000 for k, v in population.items()}
else:
    geocodes = util.french_region_and_be
    population = pd.read_csv(url_department_france).groupby('regionTrends').agg({'population': 'sum'})
    augment_population = {k: pop['population'] / 100_000 for k, pop in population.iterrows()}  # pop per 100.000
    df_hospi = util.hospi_french_region_and_be(url_hospi_france_tot, url_hospi_france_new, url_hospi_belgium, 
                                           url_department_france, util.french_region_and_be, new_hosp_in=True, 
                                           tot_hosp=True, date_begin=date_begin)
df_trends = util.create_df_trends(url_trends, list_topics, geocodes)  # TODO deal with augmented data
for k in df_hospi.keys(): # Rolling average of 7 days 
    df_hospi[k] = df_hospi[k].rolling(7, center=True).mean().dropna()
    df_trends[k] = df_trends[k].rolling(7, center=True).mean().dropna()
merged_df = {k: pd.merge(df_hospi[k], df_trends[k], left_index=True, right_index=True).dropna() for k,v in geocodes.items()}
    
scaler_generator = MinMaxScaler
dg = util.DataGenerator(merged_df, n_samples, n_forecast, target, scaler_generator=scaler_generator, scaler_type='batch',
                       augment_merge=3, augment_adjacency=util.france_region_adjacency, augment_population=augment_population,
                       predict_one=predict_one, cumsum=cumsum, data_columns=list_hosp_features)
n_features = dg.n_features
target_idx = dg.target_idx

### Model

In [6]:
if not predict_one:
    weights_loss = np.array([(1/x) for x in range(1, n_forecast+1)])
else:
    weights_loss = 1

def custom_loss_function(y_true, y_pred):
    y_true = y_true * weights_loss
    y_pred = y_pred * weights_loss
    return tf.keras.losses.mean_squared_error(y_true, y_pred)

def get_encoder_decoder(batch_input_shape):
    model = Sequential()
    #model.add(Bidirectional(LSTM(8, return_sequences=True, stateful=False), 
    #                        input_shape=(n_samples, n_features), merge_mode="ave"))
    model.add(LSTM(16, return_sequences=True, stateful=False, batch_input_shape=batch_input_shape, recurrent_dropout=0))
    model.add(LSTM(8, return_sequences=False, stateful=False))
    model.add(RepeatVector(n_forecast))  # repeat
    model.add(LSTM(8, return_sequences=True, stateful=False))  # dec
    if not predict_one:
        model.add(LSTM(16, return_sequences=True, stateful=False))  # dec
        model.add(TimeDistributed(Dense(1)))
        model.add(Reshape((n_forecast,)))
    else:
        model.add(LSTM(16, return_sequences=False, stateful=False))  # dec
        model.add(Dense(1))
        model.add(Reshape((1,)))
    model.compile(loss=custom_loss_function, optimizer='adam', metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model
    
get_encoder_decoder((1, n_samples, n_features)).output_shape

(1, 20)

## Models to beat
### MultiStepLastBaseline
This model repeats the last value of hospitalisations `n_forecast` time

In [7]:
class MultiStepLastLayer(tf.keras.Model):
    """
    repeat the last hospitalisations given as input n_forecast time
    """
    def call(self, inputs, *args, **kwargs):
        a = inputs[:, -1:, target_idx:target_idx+1]  # target of the last days
        # a = tf.where(tf.not_equal(a, 0), tf.zeros_like(a), a)
        if not predict_one:
            return tf.tile(
                a,
                [1, n_forecast, 1]   # repeat target n_forecast time
            )
        else:
            return tf.tile(a, [1, 1, 1])
        
class MultiStepLastBaseline(tf.keras.Model):
    """
    repeat the last hospitalisations given as input n_forecast time
    """
    def __init__(self, batch_input_shape, *args, **kwargs):
        super(MultiStepLastBaseline, self).__init__(name='')
        self.total = tf.Variable(initial_value=tf.zeros((1,)), trainable=False)
        self.multi_step = MultiStepLastLayer()
        self.reshape = Reshape((n_forecast,))
    
    def call(self, input_tensor, training=False):
        x = self.multi_step(input_tensor)
        return self.reshape(x)
        
    def get_weights(self):
        return None
    
    def set_weights(self, *args, **kwargs):
        return None

def get_baseline(*args, **kwargs):
    model = MultiStepLastBaseline(*args, **kwargs)
    model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

### Linear regression
linear regression of the last `n_sample` days used to predict the next `n_forecast` days

In [8]:
class LinearRegressionHospi(tf.keras.Model):
    """
    repeat the last hospitalisations given as input n_forecast time
    """
    def predict(self, inputs, *args, **kwargs):
        y = inputs[:, :, target_idx]  # target of the last days
        length = len(inputs)
        x = np.arange(n_samples).reshape(-1,1)  # dates of the target
        if not predict_one:
            result = np.zeros((length, n_forecast))
            for i in range(length):
                regr = LinearRegression().fit(x, y[i])  # linear regression of (days, target)
                result[i] = regr.predict(np.arange(n_samples, n_samples+n_forecast).reshape(-1,1))
        else:
            result = np.zeros((length, 1))
            for i in range(length):
                regr = LinearRegression().fit(x, y[i])
                result[i] = regre.predict([n_samples+n_forecast-1])
        return result
        
def get_custom_linear_regression(*args, **kwargs):
    model = LinearRegressionHospi()
    model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

### 1 fully connected layer (Dense model)
Using only the target in the fit

In [9]:
DenseModel = Sequential()
DenseModel.add(Lambda(lambda x: x[:,:,target_idx]))  # select only the target of the previous days
if not predict_one:
    DenseModel.add(Dense(n_forecast))   # predict the next target based on the previous ones
else:
    DenseModel.add(Dense(1))
DenseModel.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

def get_dense_model(batch_input_shape):
    model = Sequential()
    model.add(Lambda(lambda x: x[:,:,target_idx], batch_input_shape=batch_input_shape))  # select only the target of the previous days
    model.add(Dense(n_forecast))   # predict the next target based on the previous ones
    model.compile(loss=tf.losses.MeanSquaredError(),
                          metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

# Simple prediction
use a percentage of values for training and the remaining values for testing

In [16]:
# X% for training, remaining for test
ratio_training = 0.9
nb_datapoints = dg.batch_size
max_train = int(ratio_training * nb_datapoints)
train_idx = np.array(range(max_train))
test_idx = np.array(range(max_train, nb_datapoints))
X_train = dg.get_x(train_idx, scaled=True)
Y_train = dg.get_y(train_idx, scaled=True)
X_test_unscaled = dg.get_x(test_idx, scaled=False)
Y_test_unscaled = dg.get_y(test_idx, scaled=False)
X_test = dg.get_x(test_idx, scaled=True, use_previous_scaler=True)
Y_test = dg.get_y(test_idx, scaled=True, use_previous_scaler=True)
Y_test_unpadded_unscaled = dg.remove_padded_y(Y_test_unscaled, idx=test_idx)

### Encoder decoder

In [18]:
batch_input = len(train_idx)
encoder_decoder_train = get_encoder_decoder(batch_input_shape=(batch_input, n_samples, n_features))
encoder_decoder_train.summary()
start = time.time()
encoder_decoder_train.fit(X_train, Y_train, epochs=50, verbose=1, batch_size=batch_input)
end = time.time() - start
print(f"computed in {str(str(timedelta(seconds=end)))} s")

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (344, 30, 16)             1216      
_________________________________________________________________
lstm_5 (LSTM)                (344, 8)                  800       
_________________________________________________________________
repeat_vector_1 (RepeatVecto (344, 20, 8)              0         
_________________________________________________________________
lstm_6 (LSTM)                (344, 20, 8)              544       
_________________________________________________________________
lstm_7 (LSTM)                (344, 20, 16)             1600      
_________________________________________________________________
time_distributed_1 (TimeDist (344, 20, 1)              17        
_________________________________________________________________
reshape_1 (Reshape)          (344, 20)               

In [20]:
batch_input = len(test_idx)
encoder_decoder_prediction = get_encoder_decoder(batch_input_shape=(batch_input, n_samples, n_features) )
encoder_decoder_prediction.set_weights(encoder_decoder_train.get_weights())

Y_predicted = np.squeeze(encoder_decoder_prediction.predict(X_test, batch_size=batch_input))
Y_predicted_unscaled = dg.inverse_transform_y(Y_predicted, idx=test_idx)

print(np.mean(Y_predicted - Y_test, axis=0))
print(np.mean(Y_predicted_unscaled - Y_test_unscaled, axis=0))

Y_predicted_unpadded_unscaled =  dg.remove_padded_y(Y_predicted_unscaled, idx=test_idx)

print(np.mean(abs(Y_predicted_unpadded_unscaled - Y_test_unpadded_unscaled), axis=0))

[-0.00399999 -0.00319381 -0.00462736 -0.00495593 -0.00505185 -0.0059531
 -0.00735401 -0.00921627 -0.01118677 -0.01355288 -0.01635245 -0.01905055
 -0.02181027 -0.02537839 -0.02906431 -0.03315539 -0.03714128 -0.0412641
 -0.04614591 -0.05104901]
[-1.05436368  0.37067537  0.44653425  0.65145979  0.84681332  0.95984233
  1.00716274  1.01253387  1.00897923  0.97270614  0.89913701  0.91584237
  0.96087507  0.84870719  0.71453177  0.54581086  0.35548846  0.13264262
 -0.21607304 -0.63400138]
[ 2.33457621  3.38391714  4.27078106  5.0814401   5.9187545   6.88740913
  7.75030797  8.34271001  8.98803277  9.66452356 10.33478669 11.03699452
 11.75513771 12.45663287 13.10032979 13.78414784 14.40715156 15.10399105
 15.85113844 16.47614451]


### Baseline

In [15]:
baseline_model= get_baseline()
Y_predicted_unscaled = np.squeeze(baseline_model.predict(X_test_unscaled))

print(np.mean(Y_predicted_unscaled - Y_test_unscaled, axis=0))

Y_predicted_unpadded_unscaled =  dg.remove_padded_y(Y_predicted_unscaled, idx=test_idx)

print(np.mean(abs(Y_predicted_unpadded_unscaled - Y_test_unpadded_unscaled), axis=0))

[  0.46074878  -0.15338165  -0.40157006   0.74939613  -2.39673914
  -2.88345411   0.05736714  -0.38828503  -3.97644928  -6.80736716
  -8.93417875 -12.83595009 -16.29206925 -14.67572464 -17.6491546
 -21.76751208 -27.00664252 -30.76871981 -36.3254831  -36.26026571]
[ 97.23976213 121.77371275 121.59357122 122.7349443  128.44000302
 105.06647095  67.53846734 110.9022132  134.43473352 129.85358327
 136.07685939 141.34590485 126.69369166  95.88279133 132.96303825
 152.38068353 148.63060826 152.68435713 156.96830775 143.07008432]


### Dense model

In [18]:
batch_input_shape = (1, n_samples, n_features)
dense_model = get_dense_model(batch_input_shape)
dense_model.summary()
start = time.time()
dense_model.fit(X_train, Y_train, epochs=400, verbose=1)
end = time.time() - start
print(f"computed in {str(str(timedelta(seconds=end)))} s")

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (1, 30)                   0         
_________________________________________________________________
dense_4 (Dense)              (1, 20)                   620       
Total params: 620
Trainable params: 620
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
233/233 [==============================] - 0s 1ms/step - loss: 1.5908 - mse: 1.5908 - mae: 0.7549 - root_mean_squared_error: 1.2600
Epoch 2/400
233/233 [==============================] - 0s 1ms/step - loss: 1.0971 - mse: 1.0971 - mae: 0.6273 - root_mean_squared_error: 1.0471
Epoch 3/400
233/233 [==============================] - 0s 1ms/step - loss: 0.9083 - mse: 0.9083 - mae: 0.5514 - root_mean_squared_error: 0.9530
Epoch 4/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8290 - ms

233/233 [==============================] - 0s 1ms/step - loss: 0.7915 - mse: 0.7915 - mae: 0.5078 - root_mean_squared_error: 0.8895
Epoch 110/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8169 - mse: 0.8169 - mae: 0.5109 - root_mean_squared_error: 0.9037
Epoch 111/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8016 - mse: 0.8016 - mae: 0.5088 - root_mean_squared_error: 0.8953
Epoch 112/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7935 - mse: 0.7935 - mae: 0.5031 - root_mean_squared_error: 0.8907
Epoch 113/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7977 - mse: 0.7977 - mae: 0.5078 - root_mean_squared_error: 0.8930
Epoch 114/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7903 - mse: 0.7903 - mae: 0.5050 - root_mean_squared_error: 0.8889
Epoch 115/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7931 - mse: 0.7931 - mae: 0.5090 - root_mean_squared_error:

233/233 [==============================] - 0s 1ms/step - loss: 0.8074 - mse: 0.8074 - mae: 0.5105 - root_mean_squared_error: 0.8984
Epoch 222/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7810 - mse: 0.7810 - mae: 0.5009 - root_mean_squared_error: 0.8834
Epoch 223/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8222 - mse: 0.8222 - mae: 0.5133 - root_mean_squared_error: 0.9066
Epoch 224/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7952 - mse: 0.7952 - mae: 0.5062 - root_mean_squared_error: 0.8917
Epoch 225/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8022 - mse: 0.8022 - mae: 0.5108 - root_mean_squared_error: 0.8956
Epoch 226/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8027 - mse: 0.8027 - mae: 0.5081 - root_mean_squared_error: 0.8957
Epoch 227/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7823 - mse: 0.7823 - mae: 0.5039 - root_mean_squared_error:

233/233 [==============================] - 0s 1ms/step - loss: 0.7821 - mse: 0.7821 - mae: 0.5055 - root_mean_squared_error: 0.8843
Epoch 334/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7888 - mse: 0.7888 - mae: 0.5043 - root_mean_squared_error: 0.8880
Epoch 335/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8080 - mse: 0.8080 - mae: 0.5086 - root_mean_squared_error: 0.8988
Epoch 336/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7935 - mse: 0.7935 - mae: 0.5021 - root_mean_squared_error: 0.8905
Epoch 337/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7803 - mse: 0.7803 - mae: 0.4942 - root_mean_squared_error: 0.8828
Epoch 338/400
233/233 [==============================] - 0s 1ms/step - loss: 0.7913 - mse: 0.7913 - mae: 0.5055 - root_mean_squared_error: 0.8895
Epoch 339/400
233/233 [==============================] - 0s 1ms/step - loss: 0.8074 - mse: 0.8074 - mae: 0.5021 - root_mean_squared_error:

In [19]:
Y_predicted = np.squeeze(dense_model.predict(X_test, batch_size=1))
Y_predicted_unscaled = dg.inverse_transform_y(Y_predicted, idx=test_idx)

print(np.mean(Y_predicted - Y_test, axis=0))
print(np.mean(Y_predicted_unscaled - Y_test_unscaled, axis=0))

Y_predicted_unpadded_unscaled =  dg.remove_padded_y(Y_predicted_unscaled, idx=test_idx)

print(mean_absolute_error(Y_predicted_unpadded_unscaled, Y_test_unpadded_unscaled, multioutput="raw_values"))

NameError: name 'dense_model' is not defined

In [10]:
def plot_prediction_vs_actual(df_y_real, df_y_predicted, prediction_dates, cur_loc, horizon=1, mode=0):
    """
    plot the prediction done on a specific horizon along with the last points of data before
    """
    if not predict_one:
        if mode == 0:
            horizon_range = range(horizon, horizon+1)
        elif mode == 1:
            horizon_range = range(1, horizon+1)
    else:
        horizon_range = [1]

    df_real = df_y_real[cur_loc]
    df_pred = df_y_predicted[cur_loc]
    prediction_dates = prediction_dates[:, horizon]
    
    for horizon in horizon_range:
        fig = plt.figure(figsize=(6,3))
        expected = df_real[f"{target}(t+{horizon})"].values
        plt.plot(prediction_dates, expected, marker=".", label="True value")
        prediction = df_pred[f"{target}(t+{horizon})"].values

        if not predict_one:
            plot_label = f"Prediction horizon {horizon}"
        else:
            plot_label = f"Prediction horizon {n_forecast}"
        plt.plot(prediction_dates, prediction, marker='.', label=plot_label)

        ax = fig.axes[0]
        # set locator
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
        # set formatter
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%Y'))
        # set font and rotation for date tick labels
        plt.gcf().autofmt_xdate()
        plt.title(f"Plot true and predicted values for {cur_loc}")
        plt.legend()
        plt.grid()
        plt.show()

In [11]:
class ValidationLogger(ProgbarLogger):
    """
    compute metrics on the validation set, using a different batch size than the training set
    at the end of each epoch, the weights of the training model are used to set the validation model
    the error is computed based on the unscaled and unpadded values
    """
    
    def __init__(self, validation_model, val_batch_size, X_val, Y_val, val_idx, val_geo):
        """
        :param validation_model: validation model ready to be used
        :param X_val: X validation set
        :param Y_val: Y validation set, already unscaled and unpadded
        :param val_idx: validation idx
        :param val_geo: validation geoloc
        """
        super(ValidationLogger, self).__init__(count_mode='steps')
        self.validation_model = validation_model
        self.val_batch_size = val_batch_size
        self.X_val = X_val
        self.Y_val = Y_val
        self.val_idx = val_idx
        self.val_geo = val_geo
    
    def on_epoch_end(self, epoch, logs=None):
        """
        at the end of each epoch, compute the metrics asked on the validation set
        """
        self.validation_model.set_weights(self.model.get_weights())
        Y_pred = self.validation_model.predict(self.X_val, batch_size=self.val_batch_size)
        Y_pred = dg.inverse_transform_y(Y_pred, geo=self.val_geo, idx=self.val_idx)
        Y_pred = dg.remove_padded_y(Y_pred, geo=self.val_geo, idx=self.val_idx)
        for metric in validation_metrics:
            metric.reset_states()
            metric.update_state(Y_pred, self.Y_val)
            logs[f"val_{metric.name}"] = metric.result().numpy()  # prepend name for validation set
        super(ValidationLogger, self).on_epoch_end(epoch, logs)
        


In [22]:
def compute_error(Y_expected, Y_actual) -> pd.DataFrame:
    MAE = mean_absolute_error(Y_expected, Y_actual, multioutput="raw_values")
    MSE = mean_squared_error(Y_expected, Y_actual, multioutput="raw_values")
    values = {}
    for t in range(n_forecast):
        values[f'MAE(t+{t+1})'] = [MAE[t]]
        values[f'MSE(t+{t+1})'] = [MSE[t]]
    return pd.DataFrame(values)


def walk_forward_evaluation_untrainable(model_generator: callable, nb_fit_first: int, nb_validation: int, 
                                        nb_test: int, verbose: int = 1, *args, **kwargs) -> pd.DataFrame:
    """
    walk forward evaluation for an untrainable model
    """
    # initial index used
    max_len = dg.batch_size
    train_idx = np.arange(nb_fit_first)
    valid_idx = np.arange(nb_fit_first, nb_fit_first + nb_validation)  # validation set is simply ignored
    if nb_fit_first + nb_validation >= max_len:
        finished = True  # no test can be done
    else:
        finished = False  # a test set can be created
        test_idx = np.arange(nb_fit_first + nb_validation, min(nb_fit_first + nb_validation + nb_test, max_len))
    df_error = pd.DataFrame()
    walk = 0
    last_iter = False  # True when the last iteration is reached
    while not finished:
        X_train = dg.get_x(train_idx)  # set the scalers
        Y_train = dg.get_y(train_idx)
        X_test = dg.get_x(test_idx, geo=dg.loc_init, use_previous_scaler=True)
        Y_test_real = dg.get_y(test_idx, geo=dg.loc_init, scaled=False)
        Y_test_real = dg.remove_padded_y(Y_test_real, geo=dg.loc_init, idx=test_idx)
        
        batch_size = len(X_test)
        model_pred = model_generator(batch_input_shape=(batch_size, n_samples, n_features))
        Y_predicted = model_pred.predict(X_test, batch_size=batch_size)
        Y_predicted_real = dg.inverse_transform_y(Y_predicted, geo=dg.loc_init, idx=test_idx)
        Y_predicted_real = dg.remove_padded_y(Y_predicted_real, geo=dg.loc_init, idx=test_idx)
        
        
        # compute the error using the unpadded and unscaled data
        error = compute_error(Y_test_real, Y_predicted_real)
        name = f'walk {walk + 1}'
        error['walk'] = name
        error['nb_test_datapoints'] = len(Y_test_real)
        error['days_train'] = len(train_idx)
        error['days_valid'] = len(valid_idx)
        error['days_test'] = len(test_idx)
        error = error.set_index('walk')
        df_error = df_error.append(error)

        if verbose != 0:
            display(df_error)
        if last_iter:
            finished = True
        # indexes for next fit
        train_idx = np.arange(train_idx[-1] + 1 + nb_test)
        valid_idx += nb_test
        if test_idx[-1] + nb_test >= max_len:  # last iteration, less points can be used for the test set
            last_iter = True  # last iteration to be done
            test_idx = np.arange(test_idx[-1], max_len)
        else:
            test_idx += nb_test
        walk += 1
    return df_error


def walk_forward_evaluation(model_generator: callable, nb_fit_first: int, nb_validation: int, nb_test: int, 
                            epochs: int = 400, plot=False, verbose: int = 1,
                            return_history: bool = False, batch_size_fun: callable = None,
                            es_stop_val: bool = False, *args, **kwargs
                           ) -> Union[pd.DataFrame, List[History]]:
    """
    evaluates a model using a walk forward evaluation: multiples fit are done, each followed by at most nb_test
    to evaluate the model
    :param model_generator: function returning the model to evaluate
    :param nb_fit_first: number of datapoints used for the first fit
    :param nb_validation: number of datapoints used for each evaluation set
    :param nb_test: number of datapoints used for the test set (at most)
    :param epochs: number of epochs used on each fit
    :param callbacks: list of callbacks to use
    :param verbose: verbose level. Passed to fit and used to display the error dataframe
    :param return_history: if True, returns the list of history of each walk
    :param batch_size_fun: function used to compute the batch size based on the X_train tensor
        if not specified, default to batch_size = len(train_idx)
    """
    # initial index used
    max_len = dg.batch_size
    train_idx = np.arange(nb_fit_first)
    valid_idx = np.arange(nb_fit_first, nb_fit_first + nb_validation)
    if nb_fit_first + nb_validation >= max_len:
        finished = True  # no test can be done
    else:
        finished = False  # a test set can be created
        test_idx = np.arange(nb_fit_first + nb_validation, min(nb_fit_first + nb_validation + nb_test, max_len))
    df_error = pd.DataFrame()
    walk = 0
    last_iter = False  # True when the last iteration is reached
    all_history = []
    
    while not finished:
        X_train = dg.get_x(train_idx)
        Y_train = dg.get_y(train_idx)
        if batch_size_fun is None:
            batch_size = len(X_train)
        else:
            batch_size = batch_size_fun(X_train)
            
        if len(valid_idx) > 0:
            X_val = dg.get_x(valid_idx, geo=dg.loc_init, use_previous_scaler=True)
            Y_val_real = dg.get_y(valid_idx, geo=dg.loc_init, scaled=False)
            Y_val_real = dg.remove_padded_y(Y_val_real, geo=dg.loc_init, idx=valid_idx)
            batch_size_val = len(X_val)
            model_validation = model_generator(batch_input_shape=(batch_size_val, n_samples, n_features))
            val_log = ValidationLogger(model_validation, len(X_val), X_val, Y_val_real, valid_idx, dg.loc_init)
            if es_stop_val:
                callbacks = [val_log, EarlyStopping(monitor="val_root_mean_squared_error", patience=25)]
            else:
                callbacks = [val_log]
        else:
            callbacks = None
            
        model_train = model_generator(batch_input_shape=(batch_size, n_samples, n_features))

        history = model_train.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, callbacks=callbacks, verbose=verbose)
        
        # test only on the unaugmented data without padding
        batch_size = len(test_idx)
        # construct a new model and use the previous weights to set it
        model_pred = model_generator(batch_input_shape=(batch_size, n_samples, n_features))
        model_pred.set_weights(model_train.get_weights())
        X_test = dg.get_x(test_idx, geo=dg.loc_init, use_previous_scaler=True)
        Y_test = dg.get_y(test_idx, geo=dg.loc_init, use_previous_scaler=True)
        Y_test_real = dg.get_y(test_idx, geo=dg.loc_init, scaled=False)
        Y_test_real = dg.remove_padded_y(Y_test_real, geo=dg.loc_init, idx=test_idx)
        Y_predicted = model_pred.predict(X_test, batch_size=batch_size)
        Y_predicted_real = dg.inverse_transform_y(Y_predicted, geo=dg.loc_init, idx=test_idx)
        Y_predicted_real = dg.remove_padded_y(Y_predicted_real, geo=dg.loc_init, idx=test_idx)        
        if return_history:
            # add test metrics to the history for this walk, based on the unpadded unscaled data 
            # except for the loss
            for metric in model_train.metrics:
                metric.reset_states()
                if metric.name == 'loss':
                    metric.update_state(Y_predicted_real, Y_test_real)
                else:
                    # compute metric accross each horizon
                    for i in range(n_forecast):
                        metric.update_state(Y_predicted_real[:, i], Y_test_real[:, i])
                        history.history[f"test_{metric.name}(t+{i+1})"] = [metric.result().numpy()]
                        metric.reset_states()
                    # compute mean of metric on all horizon
                    metric.update_state(Y_predicted_real, Y_test_real)
                # prepend name for test set
                history.history[f"test_{metric.name}"] = [metric.result().numpy()]
            # add number of unpadded datapoints
            history.history['nb_test_datapoints'] = [len(Y_test_real)]
            all_history.append(history)
        if not return_history or verbose != 0:
            # compute the error using the unpadded and unscaled data
            error = compute_error(Y_test_real, Y_predicted_real)
            name = f'walk {walk + 1}'
            error['walk'] = name
            error['nb_test_datapoints'] = len(Y_test_real)
            error['days_train'] = len(train_idx)
            error['days_valid'] = len(valid_idx)
            error['days_test'] = len(test_idx)
            error = error.set_index('walk')
            df_error = df_error.append(error)
        if verbose != 0:
            display(df_error)
        if last_iter:
            finished = True
        # indexes for next fit
        train_idx = np.arange(train_idx[-1] + 1 + nb_test)
        valid_idx += nb_test
        if test_idx[-1] + nb_test >= max_len:  # last iteration, less points can be used for the test set
            last_iter = True  # last iteration to be done
            test_idx = np.arange(test_idx[-1], max_len)
        else:
            test_idx += nb_test
        walk += 1
    if not return_history:
        df_error.loc['mean'] = df_error.mean()
        
    return df_error if not return_history else all_history

In [23]:
no_div = lambda x : len(x)
df_errors = walk_forward_evaluation(get_dense_model, 250, 10, 30, epochs=300, verbose=0, batch_size_fun=no_div,
                                   es_stop_val=True)

In [46]:
df_errors = walk_forward_evaluation_untrainable(get_baseline, 250, 10, 30, verbose=0)

In [47]:
df_errors

,MAE(t+1),MSE(t+1),MAE(t+2),MSE(t+2),MAE(t+3),MSE(t+3),MAE(t+4),MSE(t+4),MAE(t+5),MSE(t+5),...,MAE(t+18),MSE(t+18),MAE(t+19),MSE(t+19),MAE(t+20),MSE(t+20),nb_test_datapoints,days_train,days_valid,days_test
walk,,,,,,,,,,,,,,,,,,,,,
walk 1,2.653414,18.376976,4.322945,52.678029,5.744572,99.592669,7.075787,152.626038,8.228779,206.738369,...,13.139682,526.430186,13.107798,537.771202,13.087922,544.791437,690.0,250.0,10.0,30.0
walk 2,2.172878,10.873261,3.472671,26.068706,4.598137,43.123129,5.665217,61.351936,6.546998,80.249630,...,15.076191,496.024723,15.651139,539.153116,16.183644,579.313899,690.0,280.0,10.0,30.0
walk 3,2.168530,9.121090,3.131470,19.516919,4.084058,32.435851,5.006418,48.270193,5.833955,66.968301,...,12.543685,312.345581,12.952381,335.639696,13.394824,358.485215,690.0,310.0,10.0,30.0
walk 4,1.947412,9.317302,3.106625,25.314141,4.145549,50.678769,5.137888,86.681698,6.157143,130.161822,...,17.865632,1171.578426,18.690683,1268.008037,19.469151,1366.899668,690.0,340.0,10.0,30.0
walk 5,3.149070,26.206336,5.512423,88.594118,7.495343,170.564193,9.111803,248.506310,10.316772,320.849713,...,18.110250,673.431590,19.579194,772.061152,21.017082,920.087802,92.0,370.0,10.0,4.0
mean,2.418261,14.778993,3.909227,42.434383,5.213532,79.278922,6.399423,119.487235,7.416729,160.993567,...,15.347088,635.962101,15.996239,690.526641,16.630525,753.915604,570.4,310.0,10.0,24.8


In [13]:
# X% for training, remaining for test
ratio_training = 0.8
nb_datapoints = dg.batch_size
max_train = int(ratio_training * nb_datapoints)
train_idx = np.array(range(max_train))
val_idx = np.array(range(max_train, nb_datapoints))
X_train = dg.get_x(train_idx, scaled=True)
Y_train = dg.get_y(train_idx, scaled=True)
X_val = dg.get_x(val_idx, scaled=True, use_previous_scaler=True)
Y_val_unscaled = dg.get_y(val_idx, geo=dg.loc_init, scaled=False, use_previous_scaler=True)
Y_val_unpadded_unscaled = dg.remove_padded_y(Y_val_unscaled, idx=val_idx, geo=dg.loc_init)

model_generator = get_dense_model

batch_size_train = len(train_idx)
batch_size_val = len(X_val)
model = model_generator(batch_input_shape=(batch_size_train, n_samples, n_features))
model_validation = model_generator(batch_input_shape=(batch_size_val, n_samples, n_features))
val_log = ValidationLogger(model_validation, len(X_val), X_val, Y_val_unpadded_unscaled, val_idx, dg.loc_init)
callbacks = [val_log, EarlyStopping(monitor='val_root_mean_squared_error', mode='min', verbose=1, patience=1)]
    
start = time.time()
history = model.fit(X_train, Y_train, batch_size=batch_size_train, epochs=200, callbacks=callbacks)
end = time.time() - start
print(f"computed in {str(str(timedelta(seconds=end)))} s")

Epoch 1/200
66/66 [==============================] - 1s 2ms/step - loss: 0.3480 - mse: 0.3480 - mae: 0.3775 - root_mean_squared_error: 0.5843 - val_mean_squared_error: 493404.6562 - val_mean_absolute_error: 311.0318 - val_root_mean_squared_error: 702.4277
Epoch 2/200
66/66 [==============================] - 0s 1ms/step - loss: 0.0654 - mse: 0.0654 - mae: 0.1591 - root_mean_squared_error: 0.2550 - val_mean_squared_error: 63255.7500 - val_mean_absolute_error: 152.8721 - val_root_mean_squared_error: 251.5070
Epoch 3/200
66/66 [==============================] - 0s 1ms/step - loss: 0.0299 - mse: 0.0299 - mae: 0.1195 - root_mean_squared_error: 0.1729 - val_mean_squared_error: 54219.2266 - val_mean_absolute_error: 148.2310 - val_root_mean_squared_error: 232.8502
Epoch 4/200
66/66 [==============================] - 0s 1ms/step - loss: 0.0222 - mse: 0.0222 - mae: 0.1068 - root_mean_squared_error: 0.1489 - val_mean_squared_error: 37672.4297 - val_mean_absolute_error: 118.3708 - val_root_mean_squ

In [14]:
hyper_parameter = {
    'hidden_1': [4, 8, 16],
    'hidden_2': [0, 4, 8, 16],
    'lr': [0.001, 0.01],
    'activation': ['relu', 'elu'],
    'reg': [lambda x: regularizers.l1(l=x), lambda x: None],
    'regw': [5e-4, 1e-3],
    'optimizer': ['Adam', 'RMSprop'],
    'losses': ['mse', 'mae', custom_loss_function],
    'scaling': [MinMaxScaler, StandardScaler],
    'batch_size_div': [1],
    'epochs': [600]
}

def get_encoder_decoder(batch_input_shape, p):
    model = Sequential()
    model.add(LSTM(p['hidden_1'], return_sequences=(p['hidden_2'] != 0), 
                   batch_input_shape=batch_input_shape, kernel_regularizer=p['reg'](p['regw'])))
    if p['hidden_2'] != 0:
        model.add(LSTM(p['hidden_2'], return_sequences=False, kernel_regularizer=p['reg'](p['regw'])))
    model.add(RepeatVector(n_forecast))  # repeat
    if p['hidden_2'] != 0:
        model.add(LSTM(p['hidden_2'], return_sequences=True, kernel_regularizer=p['reg'](p['regw'])))  # dec
    if not predict_one:
        model.add(LSTM(p['hidden_1'], return_sequences=True, kernel_regularizer=p['reg'](p['regw'])))  # dec
        model.add(TimeDistributed(Dense(1, kernel_regularizer=p['reg'](p['regw']), activation=p['activation'])))
        model.add(Reshape((n_forecast,)))
    else:
        model.add(LSTM(p['hidden_1'], return_sequences=False, kernel_regularizer=p['reg'](p['regw'])))  # dec
        model.add(Dense(1, kernel_regularizer=p['reg'](p['regw']), activation=p['activation']))
        model.add(Reshape((1,)))
    model.compile(loss=p["losses"], optimizer=p["optimizer"], metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    K.set_value(model.optimizer.learning_rate, p['lr'])
    return model
    

def talos_walk(x_train, y_train, x_val, y_val, p):
    """
    walk forward evaluation used by talos to determine the best model to keep
    """
    dg.set_scaler(p['scaling'])
    model_generator = lambda batch_input_shape: get_encoder_decoder(batch_input_shape, p)

    batch_size_fun = lambda x : int(x.shape[0] / p['batch_size_div'])
    history_list = walk_forward_evaluation(model_generator, nb_fit_first=250, nb_validation=0, nb_test=30, 
                                      epochs=p['epochs'], plot=False, verbose=0, 
                                      batch_size_fun=batch_size_fun, return_history=True, es_stop_val=True)
    # compute history to return, based on the list of history
    history = History()
    # used to trick talos to compute the right amount of round_epochs. needs to be the first entry of the dict
    # create an array of len == mean of number of epoch of each history
    history.history['ep'] = np.arange(int(np.mean([len(hist.history['loss']) for hist in history_list]))) + 1
    for log in history_list[0].history.keys():
        history.history[log] = [np.mean([hist.history[log][-1] for hist in history_list])]
    return history, model_generator((1, n_samples, n_features))

scan_object = talos.Scan(
    x=[],
    y=[],
    x_val=[],
    y_val=[],
    params=hyper_parameter,
    model=talos_walk,
    experiment_name='trends1', 
    fraction_limit=0.01
)

  0%|          | 0/23 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
analyze_object = talos.Analyze(scan_object)
print("MAE", analyze_object.low('mae'))
print("MSE", analyze_object.low('mse'))
print("RMSE", analyze_object.low('root_mean_squared_error'))
print("TEST MAE", analyze_object.low('test_mae'))
print("TEST MSE", analyze_object.low('test_mse'))
print("TEST RMSE", analyze_object.low('test_root_mean_squared_error'))
df = analyze_object.table('test_root_mean_squared_error', ascending=True)
print(df.columns)
columns = [f"test_mae(t+{i})" for i in range(1, n_forecast+1)]
df[['hidden_1', 'hidden_2', 'lr', 'batch_size_div', 'scaling', 'test_mse', 'test_mae'] + columns]

# 2 estimators combined
The first estimator should be simple and output an estimate $ \hat{y}_{(1)}(t) $ of $ y(t) $
The correction factor ($ C(t) $) between the estimation and the real value is then computed:

$$ y(t) = \hat{y}_{(1)}(t) * ( 1 + C(t)) $$

$$ C(t) = \frac{y(t) - \hat{y}_{(1)}(t)}{\hat{y}_{(1)}(t)} $$

This formula is used such that $C(t)$ should be bounded in [-1, 1] if the target is a positive value and the first estimator is relatively closed to the expected value.

Anoter estimator is then trained based on the first one. The goal is to be able to predict the correction $ C(t) $ that should be done on the first estimator, in order to reduce the estimation error and get a better prediction. The prediction of $ C(t) $ is written $ \hat{C}(t) $. Once this estimator is computed, the final prediction can be constructed:

$$ \hat{y}(t) = \hat{y}_{(1)}(t) * ( 1 + \hat{C}(t)) $$



## First estimator
Construct the predition $\hat{y}_{(1)}(t)$ and compute $C(t)$ based on it

In [13]:
# X% for training, remaining for test
ratio_training = 0.8
epochs = 200

nb_datapoints = dg.batch_size
max_train = int(ratio_training * nb_datapoints)
train_idx = np.array(range(max_train))
valid_idx = np.array([])
test_idx = np.array(range(max_train, nb_datapoints))

X_train_1 = dg.get_x(train_idx, scaled=True)
Y_train = dg.get_y(train_idx, scaled=True)

if len(valid_idx) > 0:
    X_val_1 = dg.get_x(val_idx, geo=dg.loc_init, scaled=True, use_previous_scaler=True)
    Y_val_unscaled = dg.get_y(val_idx, geo=dg.loc_init, scaled=False)
    Y_val_real = dg.remove_padded_y(Y_val_unscaled, idx=val_idx, geo=dg.loc_init)

X_test_1 = dg.get_x(test_idx, scaled=True, geo=dg.loc_init, use_previous_scaler=True)
Y_test_unscaled = dg.get_y(test_idx, scaled=False, geo=dg.loc_init)
Y_test_real = dg.remove_padded_y(Y_test_unscaled, idx=test_idx, geo=dg.loc_init)
    
"""
X_train_val = dg.get_x(train_val_idx, scaled=True)
df_Y_real = {loc: dg.df[loc][dg.target_columns].iloc[train_val_idx] for loc in dg.df}
Y_real_train_val = dg.get_y(train_val_idx, scaled=True, use_previous_scaler=True)
"""

model_generator = get_dense_model

batch_size_train = len(X_train_1)
batch_size_test = len(X_test_1)

if len(valid_idx) > 0:  # use validation set for an early stop
    batch_size_val = len(X_val_1)
    model_validation = model_generator(batch_input_shape=(batch_size_val, n_samples, dg.n_features))
    val_log = ValidationLogger(model_validation, len(X_val), X_val, Y_val_real, valid_idx, dg.loc_init)
    callbacks = [val_log, EarlyStopping(monitor='val_root_mean_squared_error', mode='min', verbose=1, patience=20)]
else:
    callbacks = None

model = model_generator(batch_input_shape=(batch_size_train, n_samples, dg.n_features))
history = model.fit(X_train_1, Y_train, batch_size=batch_size_train, epochs=epochs, callbacks=callbacks)
# compute the predictions on both the training and the validation
Y_train_pred_1 = model.predict(X_train_1, batch_size=batch_size_train)

model_prediction = model_generator(batch_input_shape=(batch_size_test, n_samples, dg.n_features))
model_prediction.set_weights(model.get_weights())
Y_test_pred_1 = model_prediction.predict(X_test_1, batch_size=batch_size_test)
# unscale and unpad the data
Y_test_pred_unscaled_1 = dg.inverse_transform_y(Y_test_pred_1, idx=test_idx, geo=dg.loc_init)
Y_test_pred_real_1 = dg.remove_padded_y(Y_test_pred_unscaled_1, idx=test_idx, geo=dg.loc_init)

"""
df_predicted_1 = dg.inverse_transform_y(Y_train_val_pred, idx=train_val_idx, return_type='dict_df')
df_predicted_1

# predictions on the training set
model_prediction = model_generator(batch_input_shape=(batch_size_train, n_samples, dg.n_features))
model_prediction.set_weights(model.get_weights())
Y_train_pred = model_prediction.predict(X_train, batch_size=batch_size_train)

# predictions on the validation without augmented regions
model_prediction = model_generator(batch_input_shape=(batch_size_val, n_samples, dg.n_features))
model_prediction.set_weights(model.get_weights())
Y_val_pred = model_prediction.predict(X_val, batch_size=batch_size_val)
df_Y_real = {loc: dg.df[loc][dg.target_columns].iloc[train_val_idx] for loc in dg.df}
"""

Epoch 1/200
1/1 [==============================] - 1s 608ms/step - loss: 0.4331 - mse: 0.4331 - mae: 0.4510 - root_mean_squared_error: 0.6581
Epoch 2/200
1/1 [==============================] - 0s 3ms/step - loss: 0.3930 - mse: 0.3930 - mae: 0.4266 - root_mean_squared_error: 0.6269
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - loss: 0.3669 - mse: 0.3669 - mae: 0.4106 - root_mean_squared_error: 0.6057
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 0.3467 - mse: 0.3467 - mae: 0.3980 - root_mean_squared_error: 0.5888
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 0.3299 - mse: 0.3299 - mae: 0.3873 - root_mean_squared_error: 0.5743
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 0.3152 - mse: 0.3152 - mae: 0.3779 - root_mean_squared_error: 0.5614
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 0.3022 - mse: 0.3022 - mae: 0.3694 - root_mean_squared_error: 0.5497
Epoch 8/200
1/1 [=

1/1 [==============================] - 0s 2ms/step - loss: 0.0325 - mse: 0.0325 - mae: 0.1232 - root_mean_squared_error: 0.1803
Epoch 118/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0322 - mse: 0.0322 - mae: 0.1228 - root_mean_squared_error: 0.1794
Epoch 119/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0319 - mse: 0.0319 - mae: 0.1221 - root_mean_squared_error: 0.1785
Epoch 120/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0316 - mse: 0.0316 - mae: 0.1217 - root_mean_squared_error: 0.1777
Epoch 121/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0313 - mse: 0.0313 - mae: 0.1211 - root_mean_squared_error: 0.1768
Epoch 122/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0310 - mse: 0.0310 - mae: 0.1206 - root_mean_squared_error: 0.1760
Epoch 123/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0307 - mse: 0.0307 - mae: 0.1201 - root_mean_squared_error: 0.1752
Epoch 124/200
1/1 [=

"\ndf_predicted_1 = dg.inverse_transform_y(Y_train_val_pred, idx=train_val_idx, return_type='dict_df')\ndf_predicted_1\n\n# predictions on the training set\nmodel_prediction = model_generator(batch_input_shape=(batch_size_train, n_samples, dg.n_features))\nmodel_prediction.set_weights(model.get_weights())\nY_train_pred = model_prediction.predict(X_train, batch_size=batch_size_train)\n\n# predictions on the validation without augmented regions\nmodel_prediction = model_generator(batch_input_shape=(batch_size_val, n_samples, dg.n_features))\nmodel_prediction.set_weights(model.get_weights())\nY_val_pred = model_prediction.predict(X_val, batch_size=batch_size_val)\ndf_Y_real = {loc: dg.df[loc][dg.target_columns].iloc[train_val_idx] for loc in dg.df}\n"

In [13]:
# compute the corrections and add them to a dataframe
data_dg_2 = [f'{topic}(t{i})' for i in range(-n_samples+1, 0, 1) for topic in list_topics] + [topic for topic in list_topics]
target_df_2 = [f'{target}(t+{i})' for i in range(1, n_forecast+1)]
df_c = {loc : dg.df[loc][data_dg_2 + target_df_2] for loc in dg.df}
"""
for k in dg.df:
    # add the data columns to the dataframe
    df_c[k] = dg.df[k][data_dg_2]
    for t in range(1, n_forecast+1):  # add the target columns
        target_t = f'{target}(t+{t})'
        df_c[k][f'C(t+{t})'] = (df_Y_real[k][target_t] - df_predicted_1[k][target_t]) / df_predicted_1[k][target_t]
"""

"\nfor k in dg.df:\n    # add the data columns to the dataframe\n    df_c[k] = dg.df[k][data_dg_2]\n    for t in range(1, n_forecast+1):  # add the target columns\n        target_t = f'{target}(t+{t})'\n        df_c[k][f'C(t+{t})'] = (df_Y_real[k][target_t] - df_predicted_1[k][target_t]) / df_predicted_1[k][target_t]\n"

In [14]:
df_c['BE']

Fièvre(t-29)  Mal de gorge(t-29)  Symptôme(t-29)  \
LOC DATE                                                           
BE  2020-03-04     25.402051           25.829284       22.034428   
    2020-03-05     25.245307           25.878495       21.941446   
    2020-03-06     24.616242           25.042280       21.586268   
    2020-03-07     24.280325           24.475947       21.198360   
    2020-03-08     24.268387           24.651690       20.792376   
...                      ...                 ...             ...   
    2021-03-17     22.274262           35.131666       22.239621   
    2021-03-18     22.250779           35.052849       22.831868   
    2021-03-19     22.082581           34.776027       23.138793   
    2021-03-20     21.799723           33.634505       23.251944   
    2021-03-21     21.560343           32.425871       23.347154   

                Fièvre(t-28)  Mal de gorge(t-28)  Symptôme(t-28)  \
LOC DATE                                                           
BE  2020-03-04     25.245307           25.878495       21.941446   
    2020-03-05     24.616242           25.042280       21.586268   
    2020-03-06     24.280325           24.475947       21.198360   
    2020-03-07     24.268387           24.651690       20.792376   
    2020-03-08     24.279004           24.994527       20.590845   
...                      ...                 ...             ...   
    2021-03-17     22.250779           35.052849       22.831868   
    2021-03-18     22.082581           34.776027       23.138793   
    2021-03-19     21.799723           33.634505       23.251944   
    2021-03-20     21.560343           32.425871       23.347154   
    2021-03-21     21.668993           31.181405       23.458170   

                Fièvre(t-27)  Mal de gorge(t-27)  Symptôme(t-27)  \
LOC DATE                                                           
BE  2020-03-04     24.616242           25.042280       21.586268   
    2020-03-05     24.280325           24.475947       21.198360   
    2020-03-06     24.268387           24.651690       20.792376   
    2020-03-07     24.279004           24.994527       20.590845   
    2020-03-08     24.030306           24.884379       20.461919   
...                      ...                 ...             ...   
    2021-03-17     22.082581           34.776027       23.138793   
    2021-03-18     21.799723           33.634505       23.251944   
    2021-03-19     21.560343           32.425871       23.347154   
    2021-03-20     21.668993           31.181405       23.458170   
    2021-03-21     21.891241           29.244671       23.651876   

                Fièvre(t-26)  ...  NEW_HOSP(t+11)  NEW_HOSP(t+12)  \
LOC DATE                      ...                                   
BE  2020-03-04     24.280325  ...       66.000000       96.428571   
    2020-03-05     24.268387  ...       96.428571      138.428571   
    2020-03-06     24.279004  ...      138.428571      184.428571   
    2020-03-07     24.030306  ...      184.428571      214.142857   
    2020-03-08     24.089472  ...      214.142857      239.857143   
...                      ...  ...             ...             ...   
    2021-03-17     21.799723  ...      254.714286      261.571429   
    2021-03-18     21.560343  ...      261.571429      274.571429   
    2021-03-19     21.668993  ...      274.571429      269.285714   
    2021-03-20     21.891241  ...      269.285714      263.285714   
    2021-03-21     21.963463  ...      263.285714      267.714286   

                NEW_HOSP(t+13)  NEW_HOSP(t+14)  NEW_HOSP(t+15)  \
LOC DATE                                                         
BE  2020-03-04      138.428571      184.428571      214.142857   
    2020-03-05      184.428571      214.142857      239.857143   
    2020-03-06      214.142857      239.857143      282.000000   
    2020-03-07      239.857143      282.000000      334.000000   
    2020-03-08      282.000000      334.000000      374.142857   
.

In [17]:
scaler_generator = MinMaxScaler
dg_2 = util.DataGenerator(df_c, n_samples, n_forecast, target=target, scaler_generator=scaler_generator, 
                          scaler_type='batch', augment_merge=0, predict_one=False, cumsum=False,
                          data_columns=[k for k in list_topics], no_lag=True)
dg_2.set_loc_init(dg.loc_init)  # consider the other localisations as being augmented

## Train the second estimator to compute $Y_2(t)$

In [17]:
X_train_2 = dg_2.get_x(train_idx, scaled=True)
Y_train_2 = dg_2.get_y(train_idx, scaled=True)

if len(valid_idx) > 0:
    X_val_2 = dg_2.get_x(valid_idx, scaled=True, use_previous_scaler=True, geo=dg_2.loc_init)
    Y_val_unscaled_2 = dg.get_y(valid_idx, geo=dg.loc_init, scaled=False)
    Y_val_real_2 = dg.remove_padded_y(Y_val_unscaled, idx=val_idx, geo=dg.loc_init)
    batch_size_val = len(X_val_2)
    
X_test_2 = dg_2.get_x(test_idx, scaled=True, use_previous_scaler=True, geo=dg_2.loc_init)
Y_test_unscaled_2 = dg.get_y(test_idx, scaled=False, geo=dg_2.loc_init)
Y_test_real_2 = dg.remove_padded_y(Y_test_unscaled_2, idx=test_idx, geo=dg_2.loc_init)

model_generator = get_dense_model

batch_size_train = len(X_train_2)
batch_size_test = len(X_test_2)
model = model_generator(batch_input_shape=(batch_size_train, n_samples, dg_2.n_features))
    
history = model.fit(X_train_2, Y_train_2, batch_size=batch_size_train, epochs=epochs, callbacks=None)
# compute the predictions on both the training and the validation
Y_train_pred_2 = model.predict(X_train_2, batch_size=batch_size_train)

model_prediction = model_generator(batch_input_shape=(batch_size_test, n_samples, dg_2.n_features))
model_prediction.set_weights(model.get_weights())
Y_test_pred_2 = model_prediction.predict(X_test_2, batch_size=batch_size_test)
# unscale and unpad the data
Y_test_pred_unscaled_2 = dg_2.inverse_transform_y(Y_test_pred_2, idx=test_idx, geo=dg_2.loc_init)
Y_test_pred_real_2 = dg.remove_padded_y(Y_test_pred_unscaled_2, idx=test_idx, geo=dg_2.loc_init)

Epoch 1/200
1/1 [==============================] - 0s 127ms/step - loss: 0.5092 - mse: 0.5092 - mae: 0.5694 - root_mean_squared_error: 0.7136
Epoch 2/200
1/1 [==============================] - 0s 3ms/step - loss: 0.4733 - mse: 0.4733 - mae: 0.5465 - root_mean_squared_error: 0.6879
Epoch 3/200
1/1 [==============================] - 0s 3ms/step - loss: 0.4491 - mse: 0.4491 - mae: 0.5306 - root_mean_squared_error: 0.6702
Epoch 4/200
1/1 [==============================] - 0s 4ms/step - loss: 0.4300 - mse: 0.4300 - mae: 0.5177 - root_mean_squared_error: 0.6557
Epoch 5/200
1/1 [==============================] - 0s 4ms/step - loss: 0.4137 - mse: 0.4137 - mae: 0.5065 - root_mean_squared_error: 0.6432
Epoch 6/200
1/1 [==============================] - 0s 3ms/step - loss: 0.3994 - mse: 0.3994 - mae: 0.4965 - root_mean_squared_error: 0.6320
Epoch 7/200
1/1 [==============================] - 0s 4ms/step - loss: 0.3865 - mse: 0.3865 - mae: 0.4874 - root_mean_squared_error: 0.6217
Epoch 8/200
1/1 [=

1/1 [==============================] - 0s 3ms/step - loss: 0.0906 - mse: 0.0906 - mae: 0.2469 - root_mean_squared_error: 0.3010
Epoch 118/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0904 - mse: 0.0904 - mae: 0.2467 - root_mean_squared_error: 0.3006
Epoch 119/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0901 - mse: 0.0901 - mae: 0.2463 - root_mean_squared_error: 0.3002
Epoch 120/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0899 - mse: 0.0899 - mae: 0.2461 - root_mean_squared_error: 0.2998
Epoch 121/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0897 - mse: 0.0897 - mae: 0.2458 - root_mean_squared_error: 0.2994
Epoch 122/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0894 - mse: 0.0894 - mae: 0.2456 - root_mean_squared_error: 0.2991
Epoch 123/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0892 - mse: 0.0892 - mae: 0.2453 - root_mean_squared_error: 0.2987
Epoch 124/200
1/1 [=

## Construct the final prediction by combining the two

In [20]:
class AssembleLayerTimeDist(tf.keras.Model):
    """
    assemble the prediction from the trends and the predictions from another model
    the same weight is used at every timestep (1 trainable parameter)
    """
    def __init__(self, input_shape):
        super(AssembleLayer, self).__init__(name='')
        self.dense_trends = tf.keras.layers.Dense(1, use_bias=False, 
                                                 kernel_constraint=tf.keras.constraints.MinMaxNorm(0.001, 0.2))
        batch_input_shape = (input_shape[0], input_shape[1], 1)
        self.time_dist = TimeDistributed(self.dense_trends, batch_input_shape=batch_input_shape)

    def call(self, input_tensor, training=False):
        x_trends = input_tensor[:, :, 1:]
        x_hosp = input_tensor[:, :, :1]
        x_trends = tf.keras.layers.Subtract()([x_trends, x_hosp])  # x_trends - x_hosp
        x_trends = self.time_dist(x_trends, training=training)  # apply simple weight
        return tf.keras.layers.Add()([x_trends, x_hosp])  # final prediction = x_hosp + (x_trends - x_hosp) * c

class AssembleLayer(tf.keras.Model):
    """
    assemble the prediction from the trends and the predictions from another model
    different weight are used at every timestep (n_forecast trainable parameter)
    """
    
    def __init__(self, batch_input_shape):
        super(AssembleLayer, self).__init__(name='')
        self.kernel = self.add_weight("kernel", shape=[1,batch_input_shape[1]], 
                                      constraint=tf.keras.constraints.MinMaxNorm(0.001, 0.2))

    """
    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                              shape=[1,input_shape[1]], constraint=tf.keras.constraints.MinMaxNorm(0.001, 0.2))
    """

    def call(self, input_tensor, training=False):
        x_trends = input_tensor[:, :, 1]
        x_hosp = input_tensor[:, :, 0]
        x_trends = tf.keras.layers.Subtract()([x_trends, x_hosp])  # x_trends - x_hosp
        x_trends = tf.multiply(x_trends, self.kernel)  # apply simple weight: (x_trends - x_hosp) * c
        # x_trends = tf.reshape(x_trends, x_hosp.shape)
        return tf.keras.layers.Add()([x_hosp, x_trends])  # final prediction = x_hosp + (x_trends - x_hosp) * c

    
def get_assemble(batch_input_shape):
    model = AssembleLayer(batch_input_shape)
    model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
    return model

In [19]:
X_train_assembled = np.stack([Y_train_pred_1, Y_train_pred_2], axis=2)
Y_train_assembled = Y_train  # output the target of the first model
X_test_assembled = np.stack([Y_test_pred_1, Y_test_pred_2], axis=2)
Y_test_assembled_real = Y_test_real

batch_size_train = len(X_train_assembled)
batch_size_test = len(X_test_assembled)

model_generator = get_assemble

model_train = model_generator(batch_input_shape=(batch_size_train, n_forecast, 2))
model_train.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
history = model_train.fit(X_train_assembled, Y_train_assembled, batch_size=batch_size_train, epochs=epochs)
Y_train_assembled_pred = model_train.predict(X_train_assembled, batch_size=batch_size_train)

model_prediction = model_generator(batch_input_shape=(batch_size_test, n_forecast, 2))
model_prediction.set_weights(model_train.get_weights())
Y_test_assembled_pred = model_prediction.predict(X_test_assembled, batch_size=batch_size_test)
# unscale and unpad the data
Y_test_assembled_pred_unscaled = dg_2.inverse_transform_y(Y_test_assembled_pred, idx=test_idx, geo=dg.loc_init)
Y_test_assembled_pred_real = dg.remove_padded_y(Y_test_assembled_pred_unscaled, idx=test_idx, geo=dg.loc_init)

Epoch 1/200
1/1 [==============================] - 0s 165ms/step - loss: 0.0253 - mse: 0.0253 - mae: 0.1184 - root_mean_squared_error: 0.1590
Epoch 2/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0214 - mse: 0.0214 - mae: 0.1060 - root_mean_squared_error: 0.1464
Epoch 3/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0214 - mse: 0.0214 - mae: 0.1059 - root_mean_squared_error: 0.1463
Epoch 4/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0214 - mse: 0.0214 - mae: 0.1058 - root_mean_squared_error: 0.1462
Epoch 5/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0213 - mse: 0.0213 - mae: 0.1057 - root_mean_squared_error: 0.1461
Epoch 6/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0213 - mse: 0.0213 - mae: 0.1056 - root_mean_squared_error: 0.1460
Epoch 7/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0213 - mse: 0.0213 - mae: 0.1055 - root_mean_squared_error: 0.1459
Epoch 8/200
1/1 [=

1/1 [==============================] - 0s 2ms/step - loss: 0.0199 - mse: 0.0199 - mae: 0.1005 - root_mean_squared_error: 0.1410
Epoch 118/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0199 - mse: 0.0199 - mae: 0.1005 - root_mean_squared_error: 0.1410
Epoch 119/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0199 - mse: 0.0199 - mae: 0.1005 - root_mean_squared_error: 0.1410
Epoch 120/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0199 - mse: 0.0199 - mae: 0.1004 - root_mean_squared_error: 0.1410
Epoch 121/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0199 - mse: 0.0199 - mae: 0.1004 - root_mean_squared_error: 0.1409
Epoch 122/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0199 - mse: 0.0199 - mae: 0.1004 - root_mean_squared_error: 0.1409
Epoch 123/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0198 - mse: 0.0198 - mae: 0.1004 - root_mean_squared_error: 0.1409
Epoch 124/200
1/1 [=

In [20]:
factors = model_prediction.weights[0].numpy().reshape(n_forecast)
for t in range(n_forecast):
    print(f'factor t+{t+1} = {factors[t]:.4f}')

factor t+1 = -0.0369
factor t+2 = 0.0094
factor t+3 = -0.0201
factor t+4 = -0.0503
factor t+5 = -0.0222
factor t+6 = 0.0086
factor t+7 = 0.0610
factor t+8 = -0.0460
factor t+9 = 0.0251
factor t+10 = 0.0206
factor t+11 = -0.0086
factor t+12 = 0.0465
factor t+13 = 0.0380
factor t+14 = -0.0035
factor t+15 = -0.0059
factor t+16 = 0.0511
factor t+17 = 0.0799
factor t+18 = -0.0015
factor t+19 = 0.1458
factor t+20 = 0.0549


In [21]:
error_1 = compute_error(Y_test_pred_real_1, Y_test_real)
error_1['model'] = 'model 1'
error_2 = compute_error(Y_test_pred_real_2, Y_test_real_2)
error_2['model'] = 'model trends'
error_assembled = compute_error(Y_test_assembled_pred_real, Y_test_assembled_real)
error_assembled['model'] = 'assembled'
pd.concat([error_1, error_2, error_assembled]).set_index('model')

,MAE(t+1),MSE(t+1),MAE(t+2),MSE(t+2),MAE(t+3),MSE(t+3),MAE(t+4),MSE(t+4),MAE(t+5),MSE(t+5),...,MAE(t+16),MSE(t+16),MAE(t+17),MSE(t+17),MAE(t+18),MSE(t+18),MAE(t+19),MSE(t+19),MAE(t+20),MSE(t+20)
model,,,,,,,,,,,,,,,,,,,,,
model 1,64.776166,8177.744198,85.544188,14000.047469,56.282541,6775.733298,92.281441,16304.983330,73.014681,12421.447863,...,171.245261,68173.437018,189.017537,79470.457678,216.303038,105515.005546,175.501428,73015.696985,199.017932,91250.922392
model trends,500.168623,448179.411747,490.743285,423056.173580,488.237584,428791.133248,481.568161,427597.078281,463.950663,402762.104273,...,462.392660,442370.553714,462.200601,440222.909615,478.446685,497911.997312,467.308769,461839.989367,468.023170,465973.809583
assembled,53.343115,5873.652453,88.448106,14906.556168,51.802158,5967.674274,78.317268,11706.270929,68.515821,11198.143498,...,181.193902,76003.897253,203.806071,91230.733219,216.146575,105285.687699,203.392486,94678.490116,208.180587,99320.488036


In [18]:
def walk_forward_assembly(model_generator_1: callable, dg_1: util.DataGenerator,
                          model_generator_2: callable, dg_2: util.DataGenerator,
                          model_generator_as: callable, 
                          nb_fit_first: int, nb_valid: int, nb_test: int, verbose: int = 1,
                          return_history: bool = False, batch_size_fun: callable = None,
                          es_stop_val: bool = False, epochs: int = 200) -> Union[pd.DataFrame, List[History]]:
    """
    evaluates a model using a walk forward evaluation: multiples fit are done, each followed by at most nb_test
    to evaluate the model
    :param model_generator_1: function returning the first model to evaluate
    :param target_1: target for the first model
    :param data_columns_1: data columns for the first model
    :param model_generator_2: function returning the second model to evaluate
    :param target_2: target for the second model
    :param data_columns_2: data columns for the second model
    :param model_generator_as: function returning the assembly model
    :param nb_fit_first: number of datapoints used for the first fit
    :param nb_valid: number of datapoints used for each validation set
    :param nb_test: number of datapoints used for the test set (at most)
    :param epochs: number of epochs used on each fit
    :param verbose: verbose level. Passed to fit and used to display the error dataframe
    :param return_history: if True, returns the list of history of each walk
    :param batch_size_fun: function used to compute the batch size based on the X_train tensor
        if not specified, default to batch_size = len(train_idx)
    """
    # initial index used
    max_len = dg.batch_size
    train_idx = np.arange(nb_fit_first)
    valid_idx = np.arange(nb_fit_first, nb_fit_first + nb_valid)
    if nb_fit_first + nb_valid >= max_len:
        finished = True  # no test can be done
    else:
        finished = False  # a test set can be created
        test_idx = np.arange(nb_fit_first + nb_valid, min(nb_fit_first + nb_valid + nb_test, max_len))
    df_error = pd.DataFrame()
    walk = 0
    last_iter = False  # True when the last iteration is reached
    all_history = []
    
    list_dg = [dg_1, dg_2]
    
    while not finished:
        X_train, Y_train, pred_train = [], [], []
        X_val, Y_val_unscaled, Y_val_real = [], [], []
        X_test, Y_test_unscaled, Y_test_real = [], [], []
        pred_test, pred_test_unscaled, pred_test_real = [], [], []
        
        # train the two first models
        for i, model_generator in enumerate([model_generator_1, model_generator_2]):
            dg_i = list_dg[i]
            X_train.append(dg_i.get_x(train_idx, scaled=True))
            Y_train.append(dg_i.get_y(train_idx, scaled=True))

            if len(valid_idx) > 0:  # validation set: validation logger and early stop if needed
                X_val.append(dg_i.get_x(valid_idx, geo=dg_i.loc_init, scaled=True, use_previous_scaler=True))
                Y_val_unscaled.append(dg_i.get_y(valid_idx, geo=dg_i.loc_init, scaled=False))
                Y_val_real.append(dg_i.remove_padded_y(Y_val_unscaled[-1], idx=valid_idx, geo=dg_i.loc_init))
                batch_size_val = len(X_val[-1])
                model_validation = model_generator(batch_input_shape=(batch_size_val, n_samples, dg_i.n_features))
                val_log = ValidationLogger(model_validation, batch_size_val, X_val[-1], Y_val_real[-1], 
                                             valid_idx, dg_i.loc_init)
                if es_stop_val:
                    callbacks = [val_log, EarlyStopping(monitor="val_root_mean_squared_error", patience=25)]
                else:
                    callbacks = [val_log]
            else:
                callbacks = None            

            # test set
            X_test.append(dg_i.get_x(test_idx, scaled=True, geo=dg_i.loc_init, use_previous_scaler=True))
            Y_test_unscaled.append(dg_i.get_y(test_idx, scaled=False, geo=dg_i.loc_init))
            Y_test_real.append(dg_i.remove_padded_y(Y_test_unscaled[-1], idx=test_idx, geo=dg_i.loc_init))

            batch_size_train = len(X_train[-1])
            batch_size_test = len(X_test[-1])

            # training of model 1
            model_train = model_generator(batch_input_shape=(batch_size_train, dg_i.n_samples, dg_i.n_features))
            history = model_train.fit(X_train[-1], Y_train[-1], batch_size=batch_size_train, epochs=epochs, 
                                      callbacks=callbacks, verbose=verbose)
            # prediction on the training set
            pred_train.append(model_train.predict(X_train[-1], batch_size=batch_size_train))
            # prediction on the test set
            model_prediction = model_generator(batch_input_shape=(batch_size_test, n_samples, dg_i.n_features))
            model_prediction.set_weights(model_train.get_weights())
            pred_test.append(model_prediction.predict(X_test[-1], batch_size=batch_size_test))
            pred_test_unscaled.append(dg_i.inverse_transform_y(pred_test[-1], idx=test_idx, geo=dg_i.loc_init))
            pred_test_real.append(dg_i.remove_padded_y(pred_test_unscaled[-1], idx=test_idx, geo=dg_i.loc_init))
        
        # train the assembly model
        X_train_as = np.stack(pred_train, axis=2)
        Y_train_as = Y_train[0]  # output the target of the first model
        X_test_as = np.stack(pred_test, axis=2)
        Y_test_as_real = Y_test_real[0]

        batch_size_train = len(X_train_as)
        batch_size_test = len(X_test_as)
        dg_as = list_dg[0]  # used for inverse scaling and unpadding

        model_train = model_generator_as(batch_input_shape=(batch_size_train, n_forecast, 2))
        model_train.compile(loss=tf.losses.MeanSquaredError(),
                              metrics=['mse', 'mae', tf.keras.metrics.RootMeanSquaredError()])
        history = model_train.fit(X_train_as, Y_train_as, batch_size=batch_size_train, epochs=epochs, 
                                  verbose=verbose)
        pred_train_as = model_train.predict(X_train_as, batch_size=batch_size_train)

        model_prediction = model_generator_as(batch_input_shape=(batch_size_test, n_forecast, 2))
        model_prediction.set_weights(model_train.get_weights())
        pred_test_as = model_prediction.predict(X_test_as, batch_size=batch_size_test)
        # unscale and unpad the data
        pred_test_as_unscaled = dg_as.inverse_transform_y(pred_test_as, idx=test_idx, geo=dg_as.loc_init)
        pred_test_as_real = dg_as.remove_padded_y(pred_test_as_unscaled, idx=test_idx, geo=dg_as.loc_init)
        
        """
        if return_history:
            # add test metrics to the history for this walk, based on the unpadded unscaled data 
            # except for the loss
            for metric in model_train.metrics:
                metric.reset_states()
                if metric.name == 'loss':
                    metric.update_state(Y_predicted_real, Y_test_real)
                else:
                    # compute metric accross each horizon
                    for i in range(n_forecast):
                        metric.update_state(Y_predicted_real[:, i], Y_test_real[:, i])
                        history.history[f"test_{metric.name}(t+{i+1})"] = [metric.result().numpy()]
                        metric.reset_states()
                    # compute mean of metric on all horizon
                    metric.update_state(Y_predicted_real, Y_test_real)
                # prepend name for test set
                history.history[f"test_{metric.name}"] = [metric.result().numpy()]
            # add number of unpadded datapoints
            history.history['nb_test_datapoints'] = [len(Y_test_real)]
            all_history.append(history)
        """
        if not return_history or verbose != 0:
            # compute the error using the unpadded and unscaled data
            for i in range(3):
                if i == 2:  # assembled model
                    error = compute_error(Y_test_as_real, pred_test_as_real)
                    name = f'walk {walk + 1} model assembled'
                    error['nb_test_datapoints'] = len(Y_test_as_real)
                else:  # two first model
                    error = compute_error(Y_test_real[i], pred_test_real[i])
                    name = f'walk {walk + 1} model {i+1}'
                    error['nb_test_datapoints'] = len(Y_test_real[i])
                error['name'] = name
                error['days_train'] = len(train_idx)
                error['days_valid'] = len(valid_idx)
                error['days_test'] = len(test_idx)
                error = error.set_index('name')
                df_error = df_error.append(error)
        
        if verbose != 0:
            display(df_error)
        if last_iter:
            finished = True
        # indexes for next fit
        train_idx = np.arange(train_idx[-1] + 1 + nb_test)
        valid_idx += nb_test
        if test_idx[-1] + nb_test >= max_len:  # last iteration, less points can be used for the test set
            last_iter = True  # last iteration to be done
            test_idx = np.arange(test_idx[-1], max_len)
        else:
            test_idx += nb_test
        walk += 1
    if not return_history:  # compute the mean across all walks
        len_df = len(df_error)
        df_error.loc['mean model 1'] = df_error.iloc[range(0, len_df, 3)].mean()
        df_error.loc['mean model 2'] = df_error.iloc[range(1, len_df, 3)].mean()
        df_error.loc['mean model as'] = df_error.iloc[range(2, len_df, 3)].mean()
    return df_error if not return_history else all_history

In [39]:
df_error = walk_forward_assembly(get_dense_model, dg, get_encoder_decoder, dg_2, get_assemble,
                     220, 30, 30, epochs=200, verbose=0)

In [40]:
df_error

,MAE(t+1),MSE(t+1),MAE(t+2),MSE(t+2),MAE(t+3),MSE(t+3),MAE(t+4),MSE(t+4),MAE(t+5),MSE(t+5),...,MAE(t+18),MSE(t+18),MAE(t+19),MSE(t+19),MAE(t+20),MSE(t+20),nb_test_datapoints,days_train,days_valid,days_test
name,,,,,,,,,,,,,,,,,,,,,
walk 1 model 1,84.359519,17172.500323,89.813538,2.042219e+04,112.391106,3.493799e+04,133.709972,5.574936e+04,162.788078,8.056094e+04,...,274.352663,2.349093e+05,266.400583,2.063037e+05,265.222337,1.836229e+05,690.0,250.0,0.0,30.0
walk 1 model 2,450.600920,364699.165079,421.811967,3.146787e+05,432.646109,3.303739e+05,437.511392,3.377634e+05,427.426859,3.221477e+05,...,402.364421,4.988358e+05,405.601952,5.181328e+05,408.784488,5.365369e+05,690.0,250.0,0.0,30.0
walk 1 model assembled,80.229933,15405.056767,94.533843,2.279574e+04,105.774797,3.109170e+04,136.053394,5.784007e+04,163.746402,8.160400e+04,...,245.472736,1.945924e+05,266.583363,2.066119e+05,235.938544,1.426686e+05,690.0,250.0,0.0,30.0
walk 2 model 1,24.286091,1180.521232,157.300306,4.594330e+04,135.788077,3.504677e+04,117.818483,2.704263e+04,208.914730,8.280504e+04,...,268.098850,1.077698e+05,222.236844,7.223126e+04,303.666583,1.367235e+05,690.0,280.0,0.0,30.0
walk 2 model 2,738.806763,973716.137758,765.629937,1.080124e+06,813.159996,1.245339e+06,842.099406,1.348647e+06,852.332266,1.382890e+06,...,826.388212,1.252091e+06,829.324643,1.258092e+06,832.707795,1.265583e+06,690.0,280.0,0.0,30.0
walk 2 model assembled,23.795614,1112.771730,150.065805,4.303688e+04,144.220197,3.821023e+04,101.586242,2.249797e+04,201.672379,7.903109e+04,...,284.723816,1.193197e+05,333.880978,1.605228e+05,393.198593,2.321580e+05,690.0,280.0,0.0,30.0
walk 3 model 1,39.497256,3147.437893,49.557288,4.749236e+03,45.528840,4.309270e+03,61.411482,7.343018e+03,80.037802,1.160291e+04,...,173.858495,5.855631e+04,194.819921,7.737171e+04,164.127304,5.723943e+04,690.0,310.0,0.0,30.0
walk 3 model 2,522.503503,371687.023222,432.838671,2.343321e+05,470.792789,2.835692e+05,516.808059,3.666708e+05,549.726193,4.306439e+05,...,483.482697,3.367390e+05,481.181985,3.356021e+05,478.756484,3.342417e+05,690.0,310.0,0.0,30.0
walk 3 model assembled,43.757012,3658.677410,46.480373,4.290812e+03,46.670262,4.463448e+03,50.326971,5.685681e+03,65.208359,8.302015e+03,...,171.004556,5.745480e+04,194.424415,7.722490e+04,183.711934,6.556798e+04,690.0,310.0,0.0,30.0


# Benchmark for all models
Run experiments for all models and save results to csv and source code of model to txt file

In [24]:
import logging
logging.getLogger('tensorflow').disabled = True  # remove warnings from tensorflow

result_dir = "../res/"

model_list_trainable = [
    get_dense_model,
    get_encoder_decoder
]

model_list_untrainable = [
    get_baseline,
    get_custom_linear_regression,
]

model_list_assemble = [
    (get_dense_model, dg, get_encoder_decoder, dg_2, get_assemble)
]

nb_fit = 250
nb_eval = 10
nb_test = 30
epochs = 300
es_stop_val = True

today_date = str(date.today())

def benchmark(model_generator, walk):
    model_name = model_generator.__name__
    print(model_name)
    df_errors = walk(model_generator, nb_fit, nb_eval, nb_test, 
                                        epochs=epochs, verbose=0, es_stop_val=es_stop_val)
    # register csv for error
    filename = f'{result_dir}{today_date}_{model_name}'
    file_csv = f'{filename}.csv'
    file_txt = f'{filename}.txt'
    
    i = 0
    while os.path.isfile(file_csv) or os.path.isfile(file_txt):
        i += 1
        file_csv = f'{filename}_{i}.csv'
        file_txt = f'{filename}_{i}.txt'
    df_errors.to_csv(file_csv)
    print(f'results saved to {file_csv}')
    
    # register source code of the model and additional infos
    info_txt = inspect.getsource(model_generator) + f'\n\nepochs = {epochs}, es_stop_val = {es_stop_val}' + \
        '\n\n' + str(dg)
    with open(file_txt, 'w') as file:
        file.write(info_txt)
    print(f'results saved to {file_txt}')

for model_generator in model_list_trainable:
    benchmark(model_generator, walk_forward_evaluation)
    
for model_generator in model_list_untrainable:
    benchmark(model_generator, walk_forward_evaluation_untrainable)

# assembly benchmark
for gen_1, dg_model_1, gen_2, dg_model_2, gen_as in model_list_assemble:
    model_name = gen_as.__name__
    print(model_name)
    df_errors = walk_forward_assembly(gen_1, dg_model_1, gen_2, dg_model_2, gen_as,
                     nb_fit, nb_eval, nb_test, epochs=epochs, verbose=0)
    # register csv for error
    filename = f'{result_dir}{today_date}_{model_name}'
    file_csv = f'{filename}.csv'
    file_txt = f'{filename}.txt'
    
    i = 0
    while os.path.isfile(file_csv) or os.path.isfile(file_txt):
        i += 1
        file_csv = f'{filename}_{i}.csv'
        file_txt = f'{filename}_{i}.txt'
    df_errors.to_csv(file_csv)
    print(f'results saved to {file_csv}')
    
    # register source code of the models and additional infos
    info_txt = 'model 1:\n\n' + inspect.getsource(gen_1) + \
        '\n\nmodel 2:\n\n' + inspect.getsource(gen_2) + \
        '\n\nmodel as:\n\n' + inspect.getsource(gen_as) + \
        f'\n\nepochs = {epochs}, es_stop_val = {es_stop_val}' + \
        '\n\ndg_1:\n' + str(dg_model_1) + \
        '\n\ndg_2:\n' + str(dg_model_2)
    with open(file_txt, 'w') as file:
        file.write(info_txt)
    print(f'results saved to {file_txt}')

get_dense_model
results saved to ../res/2021-04-19_get_dense_model.csv
results saved to ../res/2021-04-19_get_dense_model.txt
get_encoder_decoder
results saved to ../res/2021-04-19_get_encoder_decoder.csv
results saved to ../res/2021-04-19_get_encoder_decoder.txt
get_baseline
results saved to ../res/2021-04-19_get_baseline.csv
results saved to ../res/2021-04-19_get_baseline.txt
get_custom_linear_regression
results saved to ../res/2021-04-19_get_custom_linear_regression.csv
results saved to ../res/2021-04-19_get_custom_linear_regression.txt
get_assemble
results saved to ../res/2021-04-19_get_assemble.csv
results saved to ../res/2021-04-19_get_assemble.txt
